<a href="https://colab.research.google.com/github/FuadAhmed22/My_Portfolio/blob/main/CancerPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=8b11a600dd79e7c41ef29b15fb67e92ddc4616280a72c9d5daa87d437b6516e5
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [4]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


In [5]:
spark = SparkSession.builder.appName("cancer_prediction").getOrCreate()

In [6]:
data = spark.read.csv("/content/cancer-patient-data-sets.csv", header=True, inferSchema=True)

In [7]:
columns_to_drop = ["index", "Patient Id"]  # Add other columns if needed
data = data.drop(*columns_to_drop)

In [8]:
data = data.na.drop()
data.show()

+---+------+-------------+-----------+------------+--------------------+------------+--------------------+-------------+-------+-------+--------------+----------+-----------------+-------+-----------+-------------------+--------+-------------------+------------------------+-------------+---------+-------+-----+
|Age|Gender|Air Pollution|Alcohol use|Dust Allergy|OccuPational Hazards|Genetic Risk|chronic Lung Disease|Balanced Diet|Obesity|Smoking|Passive Smoker|Chest Pain|Coughing of Blood|Fatigue|Weight Loss|Shortness of Breath|Wheezing|Swal0ing Difficulty|Clubbing of Finger Nails|Frequent Cold|Dry Cough|Snoring|Level|
+---+------+-------------+-----------+------------+--------------------+------------+--------------------+-------------+-------+-------+--------------+----------+-----------------+-------+-----------+-------------------+--------+-------------------+------------------------+-------------+---------+-------+-----+
| 33|     1|            2|          4|           5|          

In [9]:
data.count()

1000

In [10]:
data.printSchema()

root
 |-- Age: integer (nullable = true)
 |-- Gender: integer (nullable = true)
 |-- Air Pollution: integer (nullable = true)
 |-- Alcohol use: integer (nullable = true)
 |-- Dust Allergy: integer (nullable = true)
 |-- OccuPational Hazards: integer (nullable = true)
 |-- Genetic Risk: integer (nullable = true)
 |-- chronic Lung Disease: integer (nullable = true)
 |-- Balanced Diet: integer (nullable = true)
 |-- Obesity: integer (nullable = true)
 |-- Smoking: integer (nullable = true)
 |-- Passive Smoker: integer (nullable = true)
 |-- Chest Pain: integer (nullable = true)
 |-- Coughing of Blood: integer (nullable = true)
 |-- Fatigue: integer (nullable = true)
 |-- Weight Loss: integer (nullable = true)
 |-- Shortness of Breath: integer (nullable = true)
 |-- Wheezing: integer (nullable = true)
 |-- Swal0ing Difficulty: integer (nullable = true)
 |-- Clubbing of Finger Nails: integer (nullable = true)
 |-- Frequent Cold: integer (nullable = true)
 |-- Dry Cough: integer (nullable = 

In [11]:
colNames = data.columns
colNames

['Age',
 'Gender',
 'Air Pollution',
 'Alcohol use',
 'Dust Allergy',
 'OccuPational Hazards',
 'Genetic Risk',
 'chronic Lung Disease',
 'Balanced Diet',
 'Obesity',
 'Smoking',
 'Passive Smoker',
 'Chest Pain',
 'Coughing of Blood',
 'Fatigue',
 'Weight Loss',
 'Shortness of Breath',
 'Wheezing',
 'Swal0ing Difficulty',
 'Clubbing of Finger Nails',
 'Frequent Cold',
 'Dry Cough',
 'Snoring',
 'Level']

In [12]:
feature_columns = data.columns[:-1]  # Exclude the label column and other unnecessary columns
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")

In [13]:
rf = RandomForestClassifier(labelCol="Level", featuresCol="features")

# Create a pipeline
pipeline = Pipeline(stages=[assembler, rf])

In [14]:
(training_data, testing_data) = data.randomSplit([0.8, 0.2], seed=42)

In [15]:
 model = pipeline.fit(training_data)

# Make predictions on the test set
predictions = model.transform(testing_data)

# Evaluate the model
evaluator = MulticlassClassificationEvaluator(labelCol="Level", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Accuracy:", accuracy)

Accuracy: 1.0


In [16]:
single_test_data = spark.createDataFrame([(33, 1, 2, 4, 4, 5, 4, 4, 2, 3, 1, 3, 1, 2, 1, 2, 1, 1, 2, 3, 2, 1,2)],
                                          ["Age", "Gender", "Air Pollution", "Alcohol use", "Dust Allergy",
                                                  "OccuPational Hazards", "Genetic Risk", "chronic Lung Disease",
                                                  "Balanced Diet", "Obesity", "Smoking", "Passive Smoker",
                                                  "Chest Pain", "Coughing of Blood", "Fatigue", "Weight Loss",
                                                  "Shortness of Breath", "Wheezing", "Swal0ing Difficulty",
                                                  "Clubbing of Finger Nails", "Frequent Cold", "Dry Cough", "Snoring"])

In [17]:
single_test_data.show()

+---+------+-------------+-----------+------------+--------------------+------------+--------------------+-------------+-------+-------+--------------+----------+-----------------+-------+-----------+-------------------+--------+-------------------+------------------------+-------------+---------+-------+
|Age|Gender|Air Pollution|Alcohol use|Dust Allergy|OccuPational Hazards|Genetic Risk|chronic Lung Disease|Balanced Diet|Obesity|Smoking|Passive Smoker|Chest Pain|Coughing of Blood|Fatigue|Weight Loss|Shortness of Breath|Wheezing|Swal0ing Difficulty|Clubbing of Finger Nails|Frequent Cold|Dry Cough|Snoring|
+---+------+-------------+-----------+------------+--------------------+------------+--------------------+-------------+-------+-------+--------------+----------+-----------------+-------+-----------+-------------------+--------+-------------------+------------------------+-------------+---------+-------+
| 33|     1|            2|          4|           4|                   5|       

In [18]:
feature_columns = single_test_data.columns

In [19]:
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features_vector")
single_test_data = assembler.transform(single_test_data)


In [20]:
prediction = model.transform(single_test_data)

In [21]:
predicted_level = prediction.select("prediction").collect()[0][0]
print("Predicted Level:", predicted_level)

Predicted Level: 0.0
